Something about the Workflow of this notebook


First need to select features from timeseries

In [ ]:
%run ../bin/features/ts_2_featuresets.py

Then translate ambient data set to features

In [ ]:
%run ../bin/features/ambient_2_fs.py

Now train model using nested xval /kfold grid search optimization

(best run as nohup)

In [1]:
%run ../bin/models/nest_xval_synth_MP.py

fold 0 ((311167, 246), (69254, 246))
fold 1 ((309240, 246), (70897, 246))
fold 2 ((316208, 246), (64094, 246))
fold 3 ((301251, 246), (79466, 246))
fold 4 ((275330, 246), (104926, 246))
   max_depth  n_estimators class_wt  threshold  wl_thresh        f1
0         10            10     None   0.278381         15  0.774462
RUN 1: psd : True >f1=0.812, n_folds                    5
max_depth                 10
n_estimators              10
class_wt                None
wl_thresh                 15
feature                  psd
stacking          horizontal
dims            [H0, H1, UP]
augment                 True
precision           0.849796
recall              0.711694
f1                  0.774462
threshold           0.278381
Name: 0, dtype: object
RUN 1 Execution time in seconds: 941.4671924114227
[{'n_folds': 5, 'max_depth': 10, 'n_estimators': 10, 'class_wt': None, 'wl_thresh': 15, 'feature': 'psd', 'stacking': 'horizontal', 'dims': ['H0', 'H1', 'UP'], 'augment': True}]


In [ ]:
pq_list

In [ ]:
from itertools import product
from collections import deque

import os
import pandas as pd
import glob
import numpy as np

import datetime
from scipy import signal
import pywt

import pyarrow as pa
import pyarrow.parquet as pq
import json

import joblib

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support, f1_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import accuracy_score

import sys
from nested_xval_utils import *

from multiprocessing import Pool, cpu_count

In [ ]:
pq_list=[os.path.join(os.path.dirname(os.getcwd()), 'data/feature_sets/',f) for f in os.listdir(os.path.join(os.path.dirname(os.getcwd()), 'data/feature_sets/'))]
#pd_list=[pd.read_parquet(pq) for pq in pq_list if ".pq" in pq]
meta_list=[read_meta(pq_fs) for pq_fs in pq_list if ".pq" in pq_fs]
meta_df=pd.DataFrame.from_records(meta_list)

##########
fs={'feature':['all','psd', 'wavelet', 'time'], 'stacking':['horizontal'], 'dims':[['H0','H1','UP']], 'augment':[True, False]}
fs={'feature':['psd'], 'stacking':['horizontal'], 'dims':[['H0','H1','UP']], 'augment':[True]}
feature_sets=[dict(zip(fs, v)) for v in product(*fs.values())]
d = {'n_folds':[5],'max_depth': [100], 'n_estimators': [100], 'class_wt':[None], 'wl_thresh':[-15,15],} #'wl_thresh':[-15, -10, -5, 0]
d = {'n_folds':[5],'max_depth': [10], 'n_estimators': [10], 'class_wt':[None], 'wl_thresh':[15],} #'wl_thresh':[-15, -10, -5, 0]
hyperp=[dict(zip(d, v)) for v in product(*d.values())]
#########

results=[]
outer_results =[]
first_tp=True
first_fp=True



#convert to rsn
#test_set=meta_df[meta_df.eq_name.isin(test_set_events)].record_number.unique()
#train_set=meta_df[meta_df.eq_name.isin(train_set_events)].record_number.unique()


In [ ]:
ambient_list= list(meta_df[meta_df.magnitude.isnull()].eq_name.unique())
event_list=meta_df[~meta_df.magnitude.isnull()].sort_values(['magnitude'], ascending=False).groupby("eq_name").count().sort_values(['station'], ascending=False).index.tolist()
full_list=ambient_list+event_list

In [ ]:
pq_list

In [ ]:
test_set_list=[]
train_set_list=[]
run_list=[]
num_runs=10
num_runs=1
#for k in np.arange(num_runs):
for k in np.arange(num_runs):
    run=k+1
    items = deque(full_list)
    items.rotate(-k)
    test_set_events=list(items)[::num_runs]
    train_set_events=list(set(full_list) - set(test_set_events))
    test_set_list.append(test_set_events)
    train_set_list.append(train_set_events)
    run_list.append(run)

In [ ]:
full_list

In [ ]:
meta_df

In [ ]:
meta_df.dtypes

In [ ]:
str_cols=['station','year','eq_name','record_number','mechanism','noise_lev']

meta_df[str_cols]

In [ ]:
meta_df[str_cols] = meta_df[str_cols].astype('|S') 

In [ ]:
meta_df

In [ ]:
meta_df.to_csv('../data/meta/fs_meta.csv', index=False)

In [ ]:
meta_df2=pd.read_csv('../data/meta/fs_meta.csv')

In [ ]:
meta_df2

In [ ]:
meta_df2=meta_df2['station'].astype('str')

In [ ]:
meta_df2.dtypes